In [1]:
%run ../functions.ipynb
%run ../pycodes/read_data.py

In [22]:
import os 

import numpy as np
import pywt as wt
import time

from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel

In [3]:
pd.options.display.max_rows = 6
pd.options.display.max_columns = 40

## Getting raw data

In [6]:
path_to_read = '../../data/raw/'
files = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path_to_read):
    for file in f:
        if '.DAT' in file:
            files.append(os.path.join(r, file))

In [7]:
df = pandas.DataFrame(columns = ['ID', 'desg','desi','desr','desz', 'Xaxis'])
for i, obj in enumerate(files):
    ID, filters, time = get_df(obj)
    df.loc[i] = [ID, filters['desg'], filters['desi'], filters['desr'], filters['desz'], time]
#df.to_pickle("./df_raw.pkl")

In [4]:
df = pd.read_pickle("../../data/df_raw.pkl")
mask = (df.desg.str.len() > 1) & (df.desi.str.len() > 1) & (df.desr.str.len() > 1) & (df.desz.str.len() > 1) 
df = df.loc[mask]
df

,ID,desg,desi,desr,desz,Xaxis
0,SN554563,"[[0.0, 18.98, 10.49], [1.985000000000582, 0.89...","[[0.03200000000651926, 8.001, 9.313], [10.9610...","[[2.0, 5.875, 2.431], [10.121000000006461, 6.9...","[[15.993000000002212, 1.763, 6.543], [29.95700...","[0.0, 1.4326868686868988, 2.8653737373737975, ..."
1,SN454964,"[[0.0, 4.359, 6.2], [4.9799999999959255, 33.82...","[[0.010999999998603016, 3.629, 6.148], [4.9959...","[[0.004000000000814907, 0.8701, 3.584], [14.21...","[[5.004000000000815, 4.406, 7.695], [24.139999...","[0.0, 1.2527575757575864, 2.505515151515173, 3..."
2,SN537864,"[[0.0, 0.4987, 4.764], [11.969000000004598, 8....","[[4.105999999999767, 1.357, 3.683], [13.090000...","[[0.008000000001629815, 3.18, 2.13], [13.07899...","[[3.0750000000043656, 1.838, 3.729], [4.137000...","[0.0, 1.1715252525252502, 2.3430505050505004, ..."
3,SN528418,"[[0.0, 4.402, 4.787], [2.0, 4.78, 2.24], [11.9...","[[0.031000000002677552, 1.92, 5.436], [2.15600...","[[0.01600000000325963, 4.441, 5.335], [10.0, 0...","[[0.06199999999807915, 8.656, 4.694], [2.18699...","[0.0, 1.5946565656565574, 3.189313131313115, 4..."
4,SN11302,"[[0.0, 7.765, 4.769], [2.997000000003027, 3.75...","[[0.023999999997613486, 7.797, 3.15], [3.04299...","[[0.008000000001629815, 1.343, 2.13], [4.09000...","[[3.0750000000043656, 6.63, 3.731], [4.1370000...","[0.0, 1.362696969696962, 2.725393939393924, 4...."
5,SN156890,"[[5.972999999998137, 1.287, 1.595], [12.855999...","[[0.0, 0.8759, 2.139], [7.961000000002969, 1.5...","[[5.989000000001397, 0.7197, 1.096], [19.96500...","[[0.023999999997613486, 0.9041, 2.2], [8.0, 1....","[0.0, 1.4323737373737404, 2.864747474747481, 4..."
6,SN713955,"[[0.0, 12.44, 10.48], [10.105999999999767, 11....","[[0.03200000000651926, 23.13, 9.326], [2.01600...","[[2.0, 4.097, 2.432], [10.121000000006461, 7.4...","[[0.9770000000062282, 32.5, 5.366], [15.993000...","[0.0, 1.5136565656566285, 3.027313131313257, 4..."
7,SN552152,"[[5.847999999998137, 0.9467, 4.678], [26.89800...","[[0.0, 16.29, 8.138], [5.875, 1.391, 4.627], [...","[[5.859000000004016, 5.592, 2.753], [18.898000...","[[18.92199999999866, 8.534, 5.954], [33.991999...","[0.0, 1.6232222222222275, 3.246444444444455, 4..."
8,SN289826,"[[2.989000000001397, 1.409, 2.584], [4.0670000...","[[12.012000000002445, 5.248, 4.52], [18.086000...","[[0.0, 2.66, 2.13], [4.082000000002154, 0.8006...","[[13.112999999997555, 1.316, 2.964], [26.01599...","[0.0, 1.5742626262626156, 3.148525252525231, 4..."
9,SN726687,"[[0.0, 1.095, 4.767], [2.997000000003027, 2.98...","[[0.023999999997613486, 21.72, 3.177], [3.0429...","[[0.008000000001629815, 24.78, 2.186], [3.0040...","[[0.046999999998661224, 23.07, 2.522], [3.0750...","[0.0, 1.281848484848469, 2.563696969696938, 3...."


## Gaussian Process
- nearly 20 hours to run

- mask filter cuts 28 objetcs (21319 - 21291)

In [ ]:
df = pd.read_pickle("../../data/df_raw.pkl")
mask = (df.desg.str.len() > 1) & (df.desi.str.len() > 1) & (df.desr.str.len() > 1) & (df.desz.str.len() > 1) 
df = df.loc[mask]

df['desg_GP'] = df.apply(lambda df: gplc(df.ID, df.desg[:,0],df.desg[:,1],df.desg[:,2]), axis=1) 
#df.to_pickle("df_GP_scikit_desg.pkl")
df['desi_GP'] = df.apply(lambda df: gplc(df.ID, df.desi[:,0],df.desi[:,1],df.desi[:,2]), axis=1) 
#df.to_pickle("df_GP_scikit_desi.pkl")
df['desr_GP'] = df.apply(lambda df: gplc(df.ID, df.desr[:,0],df.desr[:,1],df.desr[:,2]), axis=1) 
#df.to_pickle("df_GP_scikit_desr.pkl")
df['desz_GP'] = df.apply(lambda df: gplc(df.ID, df.desz[:,0],df.desz[:,1],df.desz[:,2]), axis=1) 
#df.to_pickle("df_GP_scikit_desz.pkl")
#df.to_pickle("../../data/df_GP_scikit.pkl")

### Vectorization try

In [11]:
df["desg_x"] = df.apply(lambda df: df.desg[:,0], axis = 1)
df["desg_y"] = df.apply(lambda df: df.desg[:,1], axis = 1)
df["desg_err"] = df.apply(lambda df: df.desg[:,2], axis = 1)

In [30]:
df["desi_x"] = df.apply(lambda df: df.desi[:,0], axis = 1)
df["desi_y"] = df.apply(lambda df: df.desi[:,1], axis = 1)
df["desi_err"] = df.apply(lambda df: df.desi[:,2], axis = 1)

In [31]:
df["desr_x"] = df.apply(lambda df: df.desr[:,0], axis = 1)
df["desr_y"] = df.apply(lambda df: df.desr[:,1], axis = 1)
df["desr_err"] = df.apply(lambda df: df.desr[:,2], axis = 1)

In [32]:
df["desz_x"] = df.apply(lambda df: df.desz[:,0], axis = 1)
df["desz_y"] = df.apply(lambda df: df.desz[:,1], axis = 1)
df["desz_err"] = df.apply(lambda df: df.desz[:,2], axis = 1)

In [18]:
vfunc = np.vectorize(gplc)
vfunc

In [19]:
df

,ID,desg,desi,desr,desz,Xaxis,desg_x,desg_y,desg_err
0,SN554563,"[[0.0, 18.98, 10.49], [1.985000000000582, 0.89...","[[0.03200000000651926, 8.001, 9.313], [10.9610...","[[2.0, 5.875, 2.431], [10.121000000006461, 6.9...","[[15.993000000002212, 1.763, 6.543], [29.95700...","[0.0, 1.4326868686868988, 2.8653737373737975, ...","[0.0, 1.985000000000582, 10.105999999999767, 1...","[18.98, 0.8934, 17.59, 0.2401, 4.116, 8.228, 1...","[10.49, 3.211, 25.58, 4.772, 2.991, 2.714, 15...."
1,SN454964,"[[0.0, 4.359, 6.2], [4.9799999999959255, 33.82...","[[0.010999999998603016, 3.629, 6.148], [4.9959...","[[0.004000000000814907, 0.8701, 3.584], [14.21...","[[5.004000000000815, 4.406, 7.695], [24.139999...","[0.0, 1.2527575757575864, 2.505515151515173, 3...","[0.0, 4.9799999999959255, 5.026999999994587, 1...","[4.359, 33.82, 48.55, 0.5993, 1.132, 8.722, 10...","[6.2, 18.69, 33.5, 2.656, 2.466, 9.819, 6.917,..."
2,SN537864,"[[0.0, 0.4987, 4.764], [11.969000000004598, 8....","[[4.105999999999767, 1.357, 3.683], [13.090000...","[[0.008000000001629815, 3.18, 2.13], [13.07899...","[[3.0750000000043656, 1.838, 3.729], [4.137000...","[0.0, 1.1715252525252502, 2.3430505050505004, ...","[0.0, 11.969000000004598, 13.06300000000192, 1...","[0.4987, 8.389, 3.84, 18.51, 19.24, 18.2, 14.9...","[4.764, 5.686, 2.286, 2.677, 4.819, 3.725, 1.3..."
3,SN528418,"[[0.0, 4.402, 4.787], [2.0, 4.78, 2.24], [11.9...","[[0.031000000002677552, 1.92, 5.436], [2.15600...","[[0.01600000000325963, 4.441, 5.335], [10.0, 0...","[[0.06199999999807915, 8.656, 4.694], [2.18699...","[0.0, 1.5946565656565574, 3.189313131313115, 4...","[0.0, 2.0, 11.976000000002387, 16.949000000000...","[4.402, 4.78, 1.273, 1.366, 7.404, 12.49, 5.71...","[4.787, 2.24, 9.76, 3.771, 2.293, 1.639, 4.494..."
4,SN11302,"[[0.0, 7.765, 4.769], [2.997000000003027, 3.75...","[[0.023999999997613486, 7.797, 3.15], [3.04299...","[[0.008000000001629815, 1.343, 2.13], [4.09000...","[[3.0750000000043656, 6.63, 3.731], [4.1370000...","[0.0, 1.362696969696962, 2.725393939393924, 4....","[0.0, 2.997000000003027, 4.075000000004366, 11...","[7.765, 3.751, 3.353, 20.28, 2.976, 9.289, 8.7...","[4.769, 2.591, 2.557, 5.705, 2.297, 2.657, 4.8..."
5,SN156890,"[[5.972999999998137, 1.287, 1.595], [12.855999...","[[0.0, 0.8759, 2.139], [7.961000000002969, 1.5...","[[5.989000000001397, 0.7197, 1.096], [19.96500...","[[0.023999999997613486, 0.9041, 2.2], [8.0, 1....","[0.0, 1.4323737373737404, 2.864747474747481, 4...","[5.972999999998137, 12.855999999999767, 18.840...","[1.287, 1.764, 19.36, 5.596, 7.464, 3.479, 6.5...","[1.595, 6.419, 26.12, 7.695, 3.791, 2.006, 3.4..."
6,SN713955,"[[0.0, 12.44, 10.48], [10.105999999999767, 11....","[[0.03200000000651926, 23.13, 9.326], [2.01600...","[[2.0, 4.097, 2.432], [10.121000000006461, 7.4...","[[0.9770000000062282, 32.5, 5.366], [15.993000...","[0.0, 1.5136565656566285, 3.027313131313257, 4...","[0.0, 10.105999999999767, 42.91400000000431, 4...","[12.44, 11.31, 20.41, 2.985, 10.06, 21.68, 13....","[10.48, 25.57, 15.92, 4.101, 15.03, 11.66, 7.8..."
7,SN552152,"[[5.847999999998137, 0.9467, 4.678], [26.89800...","[[0.0, 16.29, 8.138], [5.875, 1.391, 4.627], [...","[[5.859000000004016, 5.592, 2.753], [18.898000...","[[18.92199999999866, 8.534, 5.954], [33.991999...","[0.0, 1.6232222222222275, 3.246444444444455, 4...","[5.847999999998137, 26.898000000001048, 39.936...","[0.9467, 1.511, 29.75, 35.89, 4.228, 22.84, 9....","[4.678, 2.681, 12.36, 17.22, 13.83, 9.79, 6.87..."
8,SN289826,"[[2.989000000001397, 1.409, 2.584], [4.0670000...","[[12.012000000002445, 5.248, 4.52], [18.086000...","[[0.0, 2.66, 2.13], [4.082000000002154, 0.8006...","[[13.112999999997555, 1.316, 2.964], [26.01599...","[0.0, 1.5742626262626156, 3.148525252525231, 4...","[2.989000000001397, 4.067000000002736, 11.9610...","[1.409, 0.9207, 2.731, 0.717, 4.737, 0.9702, 0...","[2.584, 2.55, 5.684, 2.284, 2.637, 3.691, 1.25..."
9,SN726687,"[[0.0, 1.095, 4.767], [2.997000000003027, 2.98...","[[0.023999999997613486, 21.72, 3.177], [3

In [25]:
start = time.time()
df['desg_GP'] = vfunc(df.ID.values, df.desg_x.values ,df.desg_y.values, df.desg_err.values)
end = time.time()
print(end - start)

/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-l

/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-l

/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/fmoliveira/.local/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-l

ValueError: setting an array element with a sequence.

In [ ]:
# Demorou mais de 4 horas, ou seja não é problema da vectorization

In [23]:
start = time.time()
print("hello")
end = time.time()
print(end - start)

hello
0.0034668445587158203


In [17]:
def gplc(index,x, y, yerr, npoints = 100, n_restarts_optimizer = 100):
  
    X = x.reshape(-1, 1)  
    Xmin, Xmax = X.min(), X.max()
    vary = y.var()
    
    i = y > 0
    
    yerr = yerr[i]/y[i]
    y = np.log(y[i])
    X = X[i]

    const = ConstantKernel(vary, constant_value_bounds=(1e-5, 2*vary))
    rbf = RBF(length_scale = .5*Xmax, length_scale_bounds=(1e-05, Xmax))

    kernel = const * rbf
    
    GPR = GaussianProcessRegressor(kernel = kernel, alpha = yerr**2, n_restarts_optimizer = n_restarts_optimizer)
    
    GPR.fit(X, y)
    
    nX = np.linspace(Xmin, Xmax, npoints).reshape(-1, 1)
    
    ny, covy = GPR.predict(nX, return_cov=True)

    expy = np.exp(ny)
    cov = (expy * expy.reshape(-1, 1)) * covy
    # tempo, resultado, erro
    return nX.ravel(), expy, np.sqrt(cov.diagonal())

In [89]:
def gplc(index,x, y, yerr, npoints = 100, n_restarts_optimizer = 100):
    
    
    return GaussianProcessRegressor(kernel = ConstantKernel(y.var(), constant_value_bounds=(1e-5, 2*y.var())) * RBF(length_scale = .5*x.reshape(-1, 1).max(), length_scale_bounds=(1e-05, x.reshape(-1, 1).max())), alpha = (yerr/y)**2, n_restarts_optimizer = n_restarts_optimizer).fit(x.reshape(-1, 1), np.log(y[i])).predict(np.linspace(x.reshape(-1, 1).min(),x.reshape(-1, 1).max(), npoints).reshape(-1, 1), return_cov=True)
    
    
    
    
   
    #X = x.reshape(-1, 1)#df.loc[:, x].values.reshape(-1, 1)
    #y = y#df.loc[:, y].values
    #yerr = yerr#df.loc[:, yerr].values
    
    #Xmin, Xmax = X.min(), X.max()
    #vary = y.var()
    
    #i = y > 0
    
    #yerr = yerr[i]/y[i]
    #y = np.log(y[i])
    #X = X[i]

#     const = ConstantKernel(y.var(), constant_value_bounds=(1e-5, 2*y.var()))
#     rbf = RBF(length_scale = .5*Xmax, length_scale_bounds=(1e-05, Xmax))

    #kernel = ConstantKernel(y.var(), constant_value_bounds=(1e-5, 2*y.var())) * RBF(length_scale = .5*Xmax, length_scale_bounds=(1e-05, Xmax))
    
    #GPR = GaussianProcessRGettingegressor(kernel = ConstantKernel(y.var(), constant_value_bounds=(1e-5, 2*y.var())) * RBF(length_scale = .5*Xmax, length_scale_bounds=(1e-05, Xmax)), alpha = yerr**2, n_restarts_optimizer = n_restarts_optimizer)
    
  #AQUI  #GaussianProcessRegressor(kernel = ConstantKernel(y.var(), constant_value_bounds=(1e-5, 2*y.var())) * RBF(length_scale = .5*x.reshape(-1, 1).max(), length_scale_bounds=(1e-05, x.reshape(-1, 1).max())), alpha = (yerr/y)**2, n_restarts_optimizer = n_restarts_optimizer).fit(x.reshape(-1, 1), np.log(y[i]))
    
    #nX = np.linspace(X.min(),X.max(), npoints).reshape(-1, 1)
    
    #ny, covy = GPR.predict(np.linspace(X.min(),X.max(), npoints).reshape(-1, 1), return_cov=True)

    #expy = np.exp(ny)
    #cov = (expy * expy.reshape(-1, 1)) * covy
    # tempo, resultado, erro
    ##return GPR.predict(np.linspace(x.reshape(-1, 1).min(),x.reshape(-1, 1).max(), npoints).reshape(-1, 1), return_cov=True)
#nX.ravel(), expy, np.sqrt(cov.diagonal())


## Wavelets

In [6]:
df_GP = pd.read_pickle("../../data/df_GP_scikit.pkl")
df_GP

,ID,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP
0,SN554563,"[[0.0, 18.98, 10.49], [1.985000000000582, 0.89...","[[0.03200000000651926, 8.001, 9.313], [10.9610...","[[2.0, 5.875, 2.431], [10.121000000006461, 6.9...","[[15.993000000002212, 1.763, 6.543], [29.95700...","[0.0, 1.4326868686868988, 2.8653737373737975, ...","([0.0, 1.3517272727273086, 2.7034545454546173,...","([0.03200000000651926, 1.4642525252590228, 2.8...","([2.0, 3.412292929292957, 4.824585858585914, 6...","([15.993000000002212, 17.264141414143634, 18.5..."
1,SN454964,"[[0.0, 4.359, 6.2], [4.9799999999959255, 33.82...","[[0.010999999998603016, 3.629, 6.148], [4.9959...","[[0.004000000000814907, 0.8701, 3.584], [14.21...","[[5.004000000000815, 4.406, 7.695], [24.139999...","[0.0, 1.2527575757575864, 2.505515151515173, 3...","([0.0, 1.1215252525252062, 2.2430505050504124,...","([0.010999999998603016, 1.253707070705658, 2.4...","([0.004000000000814907, 1.24669696969773, 2.48...","([5.004000000000815, 6.206212121212938, 7.4084..."
2,SN537864,"[[0.0, 0.4987, 4.764], [11.969000000004598, 8....","[[4.105999999999767, 1.357, 3.683], [13.090000...","[[0.008000000001629815, 3.18, 2.13], [13.07899...","[[3.0750000000043656, 1.838, 3.729], [4.137000...","[0.0, 1.1715252525252502, 2.3430505050505004, ...","([0.0, 1.1417777777777784, 2.283555555555557, ...","([4.105999999999767, 5.236050505050272, 6.3661...","([0.008000000001629815, 1.1792828282844798, 2....","([3.0750000000043656, 4.214080808085176, 5.353..."
3,SN528418,"[[0.0, 4.402, 4.787], [2.0, 4.78, 2.24], [11.9...","[[0.031000000002677552, 1.92, 5.436], [2.15600...","[[0.01600000000325963, 4.441, 5.335], [10.0, 0...","[[0.06199999999807915, 8.656, 4.694], [2.18699...","[0.0, 1.5946565656565574, 3.189313131313115, 4...","([0.0, 1.453161616161665, 2.90632323232333, 4....","([0.031000000002677552, 1.6152424242450665, 3....","([0.01600000000325963, 1.5903737373769724, 3.1...","([0.06199999999807915, 1.6560303030283934, 3.2..."
4,SN11302,"[[0.0, 7.765, 4.769], [2.997000000003027, 3.75...","[[0.023999999997613486, 7.797, 3.15], [3.04299...","[[0.008000000001629815, 1.343, 2.13], [4.09000...","[[3.0750000000043656, 6.63, 3.731], [4.1370000...","[0.0, 1.362696969696962, 2.725393939393924, 4....","([0.0, 1.3621414141414037, 2.7242828282828073,...","([0.023999999997613486, 1.3862121212097753, 2....","([0.008000000001629815, 1.3701818181834458, 2....","([3.0750000000043656, 4.406636363640677, 5.738..."
...,...,...,...,...,...,...,...,...,...,...
21312,SN538509,"[[2.989000000001397, 2.453, 2.589], [4.0670000...","[[0.01599999999598367, 16.46, 3.163], [3.03499...","[[0.0, 11.75, 2.145], [2.995999999999185, 12.8...","[[0.03899999999703141, 13.7, 2.497], [3.067000...","[0.0, 1.060888888888873, 2.121777777777746, 3....","([2.989000000001397, 4.019020202021615, 5.0490...","([0.01599999999598367, 1.0764848484808809, 2.1...","([0.0, 1.0604545454545513, 2.1209090909091026,...","([0.03899999999703141, 1.099494949491995, 2.15..."
21313,SN710827,"[[0.0, 11.09, 4.807], [2.0, 14.44, 2.281], [9....","[[0.031000000002677552, 23.73, 5.457], [2.1560...","[[0.01600000000325963, 18.35, 5.349], [2.13999...","[[0.06199999999807915, 17.6, 4.704], [2.186999...","[0.0, 1.0607272727272974, 2.1214545454545948, ...","([0.0, 0.818888888888886, 1.637777777777772, 2...","([0.031000000002677552, 1.0211010101036553, 2....","([0.01600000000325963, 1.076565656568908, 2.13...","([0.06199999999807915, 1.0615656565637575, 2.0..."
21314,SN51813,"[[14.859000000004016, 11.64, 11.89], [30.91799...","[[12.984000000004016, 5.321, 4.986], [22.88599...","[[0.0, 1.53, 1.444], [14.86699999999837, 2.602...","[[2.014999999999418, 3.934, 3.402], [13.014999...","[0.0, 0.7759595959595931, 1.5519191919191861, ...","([14.859000000004016, 15.47476767677165, 16.09...","([12.984000000004016, 13.628808080812053, 14.2...","([0.0, 0.7659393939394075, 1.531878787878815, ...","([2.014999999999418, 2.7200606060600188, 3.425..."
21315,SN599115,"[[0.0, 3.229, 7.693], [10.0149999999

In [13]:
df_GP['wavelets'] = df_GP.apply(wavelets, axis = 1)
#df_GP.to_pickle("../../data/df_GP_wav.pkl")

## PCA

In [17]:
df_GP_wav = pd.read_pickle("../../data/df_GP_wav.pkl")
df_GP_wav

,ID,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets
0,SN554563,"[[0.0, 18.98, 10.49], [1.985000000000582, 0.89...","[[0.03200000000651926, 8.001, 9.313], [10.9610...","[[2.0, 5.875, 2.431], [10.121000000006461, 6.9...","[[15.993000000002212, 1.763, 6.543], [29.95700...","[0.0, 1.4326868686868988, 2.8653737373737975, ...","([0.0, 1.3517272727273086, 2.7034545454546173,...","([0.03200000000651926, 1.4642525252590228, 2.8...","([2.0, 3.412292929292957, 4.824585858585914, 6...","([15.993000000002212, 17.264141414143634, 18.5...","[15.808392550321383, 18.638950202591968, 20.91..."
1,SN454964,"[[0.0, 4.359, 6.2], [4.9799999999959255, 33.82...","[[0.010999999998603016, 3.629, 6.148], [4.9959...","[[0.004000000000814907, 0.8701, 3.584], [14.21...","[[5.004000000000815, 4.406, 7.695], [24.139999...","[0.0, 1.2527575757575864, 2.505515151515173, 3...","([0.0, 1.1215252525252062, 2.2430505050504124,...","([0.010999999998603016, 1.253707070705658, 2.4...","([0.004000000000814907, 1.24669696969773, 2.48...","([5.004000000000815, 6.206212121212938, 7.4084...","[30.643224836287317, 35.67248978201266, 39.557..."
2,SN537864,"[[0.0, 0.4987, 4.764], [11.969000000004598, 8....","[[4.105999999999767, 1.357, 3.683], [13.090000...","[[0.008000000001629815, 3.18, 2.13], [13.07899...","[[3.0750000000043656, 1.838, 3.729], [4.137000...","[0.0, 1.1715252525252502, 2.3430505050505004, ...","([0.0, 1.1417777777777784, 2.283555555555557, ...","([4.105999999999767, 5.236050505050272, 6.3661...","([0.008000000001629815, 1.1792828282844798, 2....","([3.0750000000043656, 4.214080808085176, 5.353...","[10.627291552130997, 15.676212809195041, 19.95..."
3,SN528418,"[[0.0, 4.402, 4.787], [2.0, 4.78, 2.24], [11.9...","[[0.031000000002677552, 1.92, 5.436], [2.15600...","[[0.01600000000325963, 4.441, 5.335], [10.0, 0...","[[0.06199999999807915, 8.656, 4.694], [2.18699...","[0.0, 1.5946565656565574, 3.189313131313115, 4...","([0.0, 1.453161616161665, 2.90632323232333, 4....","([0.031000000002677552, 1.6152424242450665, 3....","([0.01600000000325963, 1.5903737373769724, 3.1...","([0.06199999999807915, 1.6560303030283934, 3.2...","[7.707031309276862, 7.976811101741539, 8.12211..."
4,SN11302,"[[0.0, 7.765, 4.769], [2.997000000003027, 3.75...","[[0.023999999997613486, 7.797, 3.15], [3.04299...","[[0.008000000001629815, 1.343, 2.13], [4.09000...","[[3.0750000000043656, 6.63, 3.731], [4.1370000...","[0.0, 1.362696969696962, 2.725393939393924, 4....","([0.0, 1.3621414141414037, 2.7242828282828073,...","([0.023999999997613486, 1.3862121212097753, 2....","([0.008000000001629815, 1.3701818181834458, 2....","([3.0750000000043656, 4.406636363640677, 5.738...","[12.453097788277859, 12.699390011832127, 13.14..."
...,...,...,...,...,...,...,...,...,...,...,...
21312,SN538509,"[[2.989000000001397, 2.453, 2.589], [4.0670000...","[[0.01599999999598367, 16.46, 3.163], [3.03499...","[[0.0, 11.75, 2.145], [2.995999999999185, 12.8...","[[0.03899999999703141, 13.7, 2.497], [3.067000...","[0.0, 1.060888888888873, 2.121777777777746, 3....","([2.989000000001397, 4.019020202021615, 5.0490...","([0.01599999999598367, 1.0764848484808809, 2.1...","([0.0, 1.0604545454545513, 2.1209090909091026,...","([0.03899999999703141, 1.099494949491995, 2.15...","[8.0644130812419, 8.40742136235601, 8.70680067..."
21313,SN710827,"[[0.0, 11.09, 4.807], [2.0, 14.44, 2.281], [9....","[[0.031000000002677552, 23.73, 5.457], [2.1560...","[[0.01600000000325963, 18.35, 5.349], [2.13999...","[[0.06199999999807915, 17.6, 4.704], [2.186999...","[0.0, 1.0607272727272974, 2.1214545454545948, ...","([0.0, 0.818888888888886, 1.637777777777772, 2...","([0.031000000002677552, 1.0211010101036553, 2....","([0.01600000000325963, 1.076565656568908, 2.13...","([0.06199999999807915, 1.0615656565637575, 2.0...","[19.230664003880218, 21.32820583746352, 22.909..."
21314,SN51813,"[[14.859000000004016, 11.64, 11.89], [30.91799...","[[12.984000000004016, 5.321, 4.986], [22.88599...","[[0.0, 1.53, 1.444], [14.8669999999983

In [18]:
# TODO
# Da pra dar uma melhorada nesse codigo aqui ne Felipe?
## Ass: Felipe 15/09/2020
s = df_GP_wav['wavelets']
rslt = np.array(s.tolist())
print(type(rslt))
print(rslt.shape)

pca = PCA(n_components=20) #CHANGE HERE THE NUMBER OF COMPONENTS YOU WANT
pca.fit(rslt)  
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
features = pca.fit_transform(rslt)
print(features.shape)

df_features = pd.DataFrame(features, columns=['f1', 'f2','f3', 'f4','f5',
                                              'f6', 'f7','f8', 'f9','f10',
                                              'f11', 'f12','f13', 'f14','f15',
                                              'f16', 'f17','f18', 'f19','f20'])
df_features

<class 'numpy.ndarray'>
(21291, 1600)
[7.75080477e-01 1.01425485e-01 4.61381626e-02 2.05184206e-02
 1.25061675e-02 9.63334368e-03 7.78383899e-03 6.93479216e-03
 3.43213957e-03 3.21499240e-03 2.15082714e-03 1.72901197e-03
 1.40127624e-03 1.14066078e-03 1.10067200e-03 9.20854977e-04
 7.70792006e-04 6.06373927e-04 5.74896303e-04 4.24922283e-04]
[611412.03415977 221174.02953809 149173.17425849  99479.26030735
  77664.52751964  68163.06501515  61271.35792703  57833.21134671
  40685.81814524  39377.71932654  32207.96882184  28877.49939296
  25996.95377096  23455.18368534  23040.3749391   21074.44776892
  19280.98356476  17101.3676223   16651.57591156  14315.7905789 ]
(21291, 20)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,-152.456150,116.958067,77.435307,13.471403,-51.955130,-18.168116,64.503646,28.600517,-27.774363,-8.120098,-20.397438,20.430515,-69.650398,75.310421,117.447984,-41.083397,-49.527872,19.079251,17.427973,1.111995
1,-312.552895,6.881269,-117.104000,115.002821,9.379711,-139.815067,23.009522,-95.772257,-71.053158,-89.069248,-6.423624,-40.945593,38.165859,-17.001822,7.106318,-35.068364,-10.663220,2.622633,-12.197107,-2.859250
2,-153.247121,140.551473,35.816624,-1.885185,-26.144828,70.170131,-56.272975,-69.939925,72.466017,-20.776364,39.708823,-1.412278,17.577747,3.413035,14.138187,-37.382939,24.262485,-31.411977,-7.438057,18.769267
3,-599.929119,227.477497,21.354422,-4.192825,18.183494,-26.685172,63.081845,-44.028596,-6.045128,-34.436630,5.570809,-36.611397,11.885965,1.445781,19.149772,18.009616,11.514932,-26.093907,9.569982,27.505480
4,261.736801,-354.162164,197.847794,106.060450,-148.757143,-143.721834,6.856987,151.184004,-30.020039,-95.881641,-20.564531,49.183148,-56.571220,9.639244,155.164602,-37.232036,-11.203544,-8.507173,27.460559,-19.366518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21286,-610.902075,316.496196,-51.323189,26.281654,55.307812,-43.629292,119.323734,-3.941033,12.406115,-2.677448,-27.738459,20.515848,12.370646,8.089545,5.332065,18.043878,18.107453,18.263960,10.762908,-5.738165
21287,-385.137025,328.857639,-82.409956,-8.577766,75.881878,-25.120229,108.442031,16.081276,39.783495,6.492771,8.104196,-5.271721,39.555147,-18.821726,-40.400195,36.065939,4.211535,-18.706366,7.470027,20.365780
21288,476.795587,-1067.956742,-360.601472,-232.028136,-174.974045,-144.276123,-483.082741,102.094460,126.465358,-183.045258,-47.553118,-106.139239,-126.098258,71.861387,109.377995,-149.822946,168.444144,182.912008,-49.802557,-94.981275
21289,-612.440524,126.730122,-66.678784,-27.158989,27.419128,-56.102780,8.534456,-1.585473,3.247079,-47.549219,-12.443284,-11.492505,-14.476083,-9.730008,20.852124,-41.093223,1.236323,26.710109,16.355968,-23.481705


In [28]:
df_GP_wav_PCA20 = pd.concat([df_GP_wav.reset_index().drop(columns = ['index']), df_features], axis = 1, sort = True)
#df_GP_wav_PCA20.to_pickle("../../data/df_GP_wav.pkl")

In [29]:
df_GP_wav_PCA20

,ID,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,f1,f2,f3,f4,...,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,SN554563,"[[0.0, 18.98, 10.49], [1.985000000000582, 0.89...","[[0.03200000000651926, 8.001, 9.313], [10.9610...","[[2.0, 5.875, 2.431], [10.121000000006461, 6.9...","[[15.993000000002212, 1.763, 6.543], [29.95700...","[0.0, 1.4326868686868988, 2.8653737373737975, ...","([0.0, 1.3517272727273086, 2.7034545454546173,...","([0.03200000000651926, 1.4642525252590228, 2.8...","([2.0, 3.412292929292957, 4.824585858585914, 6...","([15.993000000002212, 17.264141414143634, 18.5...","[15.808392550321383, 18.638950202591968, 20.91...",-152.456150,116.958067,77.435307,13.471403,...,-18.168116,64.503646,28.600517,-27.774363,-8.120098,-20.397438,20.430515,-69.650398,75.310421,117.447984,-41.083397,-49.527872,19.079251,17.427973,1.111995
1,SN454964,"[[0.0, 4.359, 6.2], [4.9799999999959255, 33.82...","[[0.010999999998603016, 3.629, 6.148], [4.9959...","[[0.004000000000814907, 0.8701, 3.584], [14.21...","[[5.004000000000815, 4.406, 7.695], [24.139999...","[0.0, 1.2527575757575864, 2.505515151515173, 3...","([0.0, 1.1215252525252062, 2.2430505050504124,...","([0.010999999998603016, 1.253707070705658, 2.4...","([0.004000000000814907, 1.24669696969773, 2.48...","([5.004000000000815, 6.206212121212938, 7.4084...","[30.643224836287317, 35.67248978201266, 39.557...",-312.552895,6.881269,-117.104000,115.002821,...,-139.815067,23.009522,-95.772257,-71.053158,-89.069248,-6.423624,-40.945593,38.165859,-17.001822,7.106318,-35.068364,-10.663220,2.622633,-12.197107,-2.859250
2,SN537864,"[[0.0, 0.4987, 4.764], [11.969000000004598, 8....","[[4.105999999999767, 1.357, 3.683], [13.090000...","[[0.008000000001629815, 3.18, 2.13], [13.07899...","[[3.0750000000043656, 1.838, 3.729], [4.137000...","[0.0, 1.1715252525252502, 2.3430505050505004, ...","([0.0, 1.1417777777777784, 2.283555555555557, ...","([4.105999999999767, 5.236050505050272, 6.3661...","([0.008000000001629815, 1.1792828282844798, 2....","([3.0750000000043656, 4.214080808085176, 5.353...","[10.627291552130997, 15.676212809195041, 19.95...",-153.247121,140.551473,35.816624,-1.885185,...,70.170131,-56.272975,-69.939925,72.466017,-20.776364,39.708823,-1.412278,17.577747,3.413035,14.138187,-37.382939,24.262485,-31.411977,-7.438057,18.769267
3,SN528418,"[[0.0, 4.402, 4.787], [2.0, 4.78, 2.24], [11.9...","[[0.031000000002677552, 1.92, 5.436], [2.15600...","[[0.01600000000325963, 4.441, 5.335], [10.0, 0...","[[0.06199999999807915, 8.656, 4.694], [2.18699...","[0.0, 1.5946565656565574, 3.189313131313115, 4...","([0.0, 1.453161616161665, 2.90632323232333, 4....","([0.031000000002677552, 1.6152424242450665, 3....","([0.01600000000325963, 1.5903737373769724, 3.1...","([0.06199999999807915, 1.6560303030283934, 3.2...","[7.707031309276862, 7.976811101741539, 8.12211...",-599.929119,227.477497,21.354422,-4.192825,...,-26.685172,63.081845,-44.028596,-6.045128,-34.436630,5.570809,-36.611397,11.885965,1.445781,19.149772,18.009616,11.514932,-26.093907,9.569982,27.505480
4,SN11302,"[[0.0, 7.765, 4.769], [2.997000000003027, 3.75...","[[0.023999999997613486, 7.797, 3.15], [3.04299...","[[0.008000000001629815, 1.343, 2.13], [4.09000...","[[3.0750000000043656, 6.63, 3.731], [4.1370000...","[0.0, 1.362696969696962, 2.725393939393924, 4....","([0.0, 1.3621414141414037, 2.7242828282828073,...","([0.023999999997613486, 1.3862121212097753, 2....","([0.008000000001629815, 1.3701818181834458, 2....","([3.0750000000043656, 4.406636363640677, 5.738...","[12.453097788277859, 12.699390011832127, 13.14...",261.736801,-354.162164,197.847794,106.060450,...,-143.721834,6.856987,151.184004,-30.020039,-95.881641,-20.564531,49.183148,-56.571220,9.639244,155.164602,-37.232036,-11.203544,-8.507173,27.460559,-19.366518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21286,SN538509,"[[2.989000000001397, 2.453, 2.589], [4.067

## Concat with redshifts and types

In [33]:
df_redshift = pd.read_pickle("../../data/df_redshift.pkl")
df_redshift

,type,type_bool,REDSHIFT_SPEC
ID,,,
SN10001,II,False,0.59162
SN100023,II,False,0.79264
SN10005,Ia,True,1.08327
SN10007,Ia,True,0.82331
SN100085,II,False,0.65600
...,...,...,...
SN99871,II,False,0.93819
SN99899,II,False,0.65243
SN99921,II,False,0.69650


In [40]:
df = pd.concat([df_GP_wav_PCA20.set_index('ID'), df_redshift], axis = 1 , join = "inner", sort = 'True')
#df.to_pickle("../../data/df.pkl")

In [41]:
df

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,f1,f2,f3,f4,f5,...,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SN10001,"[[0.0, 2.326, 4.792], [2.0, 4.617, 2.249], [9....","[[0.031000000002677552, 8.672, 5.439], [2.1560...","[[0.01600000000325963, 8.834, 5.339], [2.13999...","[[0.06199999999807915, 6.026, 4.693], [2.18699...","[0.0, 1.1313535353535435, 2.262707070707087, 3...","([0.0, 0.818888888888886, 1.637777777777772, 2...","([0.031000000002677552, 1.141676767679467, 2.2...","([0.01600000000325963, 1.076565656568908, 2.13...","([0.06199999999807915, 1.1927272727253795, 2.3...","[9.251771363349247, 8.972870155014132, 8.79160...",-627.606441,282.667898,-58.310795,25.674751,60.694436,...,-16.436875,-11.941724,-23.881895,29.151572,12.431821,6.048710,-11.165995,23.954122,18.206973,6.125696,9.448978,-0.078004,II,False,0.59162
SN100023,"[[6.984000000004016, 5.679, 6.387], [7.9920000...","[[7.00800000000163, 4.698, 7.145], [8.02300000...","[[0.0, 16.04, 17.59], [6.992000000005646, 19.8...","[[4.121000000006461, 18.01, 3.601], [7.0230000...","[0.0, 1.3437474747475018, 2.6874949494950036, ...","([6.984000000004016, 8.256929292933277, 9.5298...","([7.00800000000163, 8.189737373738987, 9.37147...","([0.0, 0.8994242424242942, 1.7988484848485884,...","([4.121000000006461, 5.423121212127635, 6.7252...","[12.22728787505526, 11.357060718395505, 10.716...",-504.040778,312.171082,-78.632844,41.877099,42.770978,...,33.056277,17.795268,-55.479337,46.750660,41.486147,33.710197,-11.581964,28.655932,19.743614,4.619543,37.266792,-16.596040,II,False,0.79264
SN10005,"[[10.027000000001863, 4.996, 8.081], [33.94500...","[[0.015000000006693881, 0.2977, 2.206], [18.94...","[[0.0, 1.033, 1.325], [13.015000000006694, 0.4...","[[0.03500000000349246, 1.826, 3.029], [48.875,...","[0.0, 1.0381919191919462, 2.0763838383838924, ...","([10.027000000001863, 10.782565656567513, 11.5...","([0.015000000006693881, 1.0530404040470573, 2....","([0.0, 1.0281717171717606, 2.056343434343521, ...","([0.03500000000349246, 1.0222929292964054, 2.0...","[2.000004411755352, 2.000003547336676, 2.00000...",-750.982205,121.777397,-119.276362,-41.573979,3.954831,...,35.188458,2.706010,-42.582112,-14.480661,36.831220,-26.554909,15.768490,16.436792,17.114795,-20.292951,-24.222566,-11.295562,Ia,True,1.08327
SN10007,"[[0.0, 6.266, 10.48], [10.105999999999767, 17....","[[0.03200000000651926, 7.161, 9.313], [2.01600...","[[2.0, 1.24, 2.429], [10.121000000006461, 20.0...","[[0.9770000000062282, 0.6586, 5.329], [15.9930...","[0.0, 1.5136565656566285, 3.027313131313257, 4...","([0.0, 1.5132626262626456, 3.0265252525252913,...","([0.03200000000651926, 1.5451717171782005, 3.0...","([2.0, 3.49314141414145, 4.9862828282829, 6.47...","([0.9770000000062282, 2.480787878794107, 3.984...","[11.341091517194192, 8.812204685737349, 6.9798...",-178.038442,319.327490,-47.013389,-53.382360,117.701566,...,21.634422,-4.749400,-62.733676,40.279861,143.184298,-99.877031,21.670852,24.155364,-26.622112,-27.616185,-19.981432,49.299250,Ia,True,0.82331
SN100085,"[[0.0, 14.24, 18.69], [0.046999999998661224, 1...","[[9.245999999999185, 2.122, 2.955], [17.238000...","[[0.008000000001629815, 0.4534, 9.048], [23.03...","[[0.024000000004889444, 8.451, 7.696], [10.168...","[0.0, 1.2024545454545972, 2.4049090909091944, ...","([0.0, 0.9405757575757411, 1.8811515151514822,...","([9.245999999999185, 10.345121212120416, 11.44...","([0.008000000001629815, 1.0792222222238468, 2....","([0.024000000004889444, 1.226212121217013, 2.4...","[15.77257266262751, 14.440593141555347, 13.455...",-366.004722,-128.508261,-147.683327,83.268745,15.517091,...,-92.039728,-126.973711,21.413424,8.652732,28.633994,107.945625,50.144762,-51.038301,-16.732577,34.820195,18.227370,40.000107,II,False,0.65600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN99871,"[[5.881999999997788, 8.155,